# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import sqlite3
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# Load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine)

X = df["message"]  # Feature: the text messages
y = df.iloc[:, 5:]  # Target labels: category columns

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):

    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word).strip() for word in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [6]:
df

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
8,8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
for i, col in enumerate(y.columns):  
    print(f"Classification Report for Category: {col}\n")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

Classification Report for Category: related

             precision    recall  f1-score   support

          0       0.59      0.36      0.45      1196
          1       0.83      0.92      0.87      4014
          2       0.44      0.32      0.37        34

avg / total       0.77      0.79      0.77      5244

Classification Report for Category: request

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4300
          1       0.82      0.37      0.51       944

avg / total       0.87      0.87      0.85      5244

Classification Report for Category: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5217
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5244

Classification Report for Category: aid_related

             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3000
          1  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {
    "clf__estimator__n_estimators": [10, 12],  
    "clf__estimator__min_samples_split": [3, 4]  
}

cv = GridSearchCV(pipeline, parameters, cv=2, verbose=2, n_jobs=-1)
cv.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=12 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=12 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=12 
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=12 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, total= 3.4min
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=10, total= 3.4min


[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  4.2min remaining: 12.5min


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, total= 3.5min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, total= 3.5min
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=12, total= 3.8min
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=12, total= 3.8min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=12, total= 4.0min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=12, total= 4.0min


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:  4.4min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 12], 'clf__estimator__min_samples_split': [3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [15]:
print(cv.best_params_)

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 12}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(X_test)

# Print classification report for each category
for i, column in enumerate(y_test.columns):
    print(f"Category: {column}\n")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("-" * 60)

Category: related

             precision    recall  f1-score   support

          0       0.63      0.31      0.42      1196
          1       0.82      0.94      0.88      4014
          2       0.48      0.38      0.43        34

avg / total       0.77      0.80      0.77      5244

------------------------------------------------------------
Category: request

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4300
          1       0.81      0.43      0.57       944

avg / total       0.87      0.88      0.86      5244

------------------------------------------------------------
Category: offer

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5217
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5244

------------------------------------------------------------
Category: aid_related

             precision    recall  f1-scor

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [17]:
import pickle

with open("classifier.pkl", "wb") as f:
    pickle.dump(pipeline, f)

In [19]:
import joblib

joblib.dump(pipeline, "classifier.pkl", compress=3)

['classifier.pkl']

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.